# Part 1: Part-of-speech tagging with HMMs

In this part of the assignment, we'll train a Hidden Markov Model (HMM) as a part-of-speech (POS) tagger, and implement both Forward-Backward inference and Viterbi decoding.

In particular:
- **(a)** Use the portion of the Penn Treebank available in the NLTK to estimate the transition and emission probabilities.
- **(b)** Implement the Forward-Backward algorithm for marginal inference of $ P(y_i\ |\ x) $
- **(c)** Implement the Viterbi algorithm to find the most likely tag _sequence_ $ \hat{y} = \arg\max_{y'} P(y'\ |\ x) $

**Note:** in the interest of a shorter assignment and giving you more time to work on projects, we've implemented **(a)** the training code and part of **(b)** the `backward()` algorithm. Do look over the solutions for those parts, as they're a good guide to the rest of your implementation!

You may want to review the Week 7 and Week 8 async, as well as the slides on part-of-speech tagging which this assignment will follow closely.

## Inspect NLTK/Penn Treebank
Before continuing, let's take a few moments to inspect the format of the training data.  The Treebank Reader object has a `tagged_sents()` function that returns an list of sentences.  Each sentence consists of a list of (word, part of speech) tuples.

In [1]:
import unittest
from importlib import reload

from w266_common import utils
import pos
import pos_test
import nltk

# Use the sample of the Penn Treebank included with NLTK.
assert(nltk.download('treebank'))
corpus = nltk.corpus.treebank

print('There are %d sentences in the corpus.' % len(corpus.tagged_sents()))
print('The first sentence is:')
corpus.tagged_sents()[0]

[nltk_data] Downloading package treebank to
[nltk_data]     /home/yeunghoman/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
There are 3914 sentences in the corpus.
The first sentence is:


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

## Part (a): HMM Parameterization

![HMM diagram](HMM.png)

Recall that a Hidden Markov Model models a sequence $[x_0, x_1, ..., x_n]$ as a Markov chain of _hidden_ states $[y_0, y_1, ..., y_n]$ and associated emissions $y_i \to x_i$ at each position. Formally, we have three sets of parameters:

1. An initial state probability $P(y_0)$ which determines the start state of the sequence.
2. Transition probabilities $P(y_i\ |\ y_{i-1})$ from one state to the next.
3. Emission probabilities $P(x_i\ |\ y_i)$ to generate an output at each timestep.

For POS tagging, we treat the word (tokens) as the observed nodes $x_i$, and the part-of-speech tags as the hidden states $y_i$ associated with each token. At training time, since the data is fully tagged, we get to observe _both_ $x_i$ and $y_i$, and so we can train our model by maximum likelihood estimation.

Recalling our n-gram models from Assignment 3, we can obtain the maximum likelihood parameters by simply counting. We'll use $t$ to denote a specific part-of-speech tag (from a tagset $T$), and $w$ to denote a specific word type (from a vocabulary $V$):

1. Initial: $ P(y_0 = t) = \frac{c(y_0 = t)}{\sum_{t' \in T} c(y_0 = t')} $
2. Transition: $ P(y_i = t\ |\ y_{i-1} = t_1) = \frac{c(t_1t)}{\sum_{t' \in T}c(t_1t')} $
3. Emission: $ P(x_i = w\ |\ y_i = t) = \frac{c(w\ |\ t)}{\sum_{w' \in V} c(w'\ |\ t)} $

### Questions
1.  Open `pos.py` and read `estimate_probabilities()` and `compute_logprobs()`.  This code should be very familiar as it's nearly identical to what you implemented in A3 to build an n-gram model.

In [2]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_estimate_probabilities"])

test_estimate_probabilities (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## Part (b): Forward-Backward

The forward-backward algorithm allows us to efficiently compute the most likely tag for any (or every) individual word. Formally, at each position $i$ we want to compute the marginal distribution $p(y_i\ |\ x_0, x_1, x_2, \cdots, x_n) = p(y_i\ |\ x)$. Note that taking the most likely tag from this will not necessarily find the most likely _sequence_ of tags - we'll tackle that in part(c) with Viterbi.



In [Week 7](https://docs.google.com/presentation/d/1DqZi3qLMlDxA-NDXiBRh34pPz2JK9hLYNcTNxDe39pk/edit#slide=id.g1eb138b3b3_1_72), we found that by applying Bayes rule and decomposing (for dynamic programming speedup), $p(y_i\ |\ x)$ can be computed with the following equations. Let $n$ be the length of the sentence, and $w_i$ be the (fixed) token at position $i = 0, 1, ..., n$:

- $ \alpha(0, t) = p(t) \times p(x_0\ |\ t) $
- $ \alpha(i, t) = p(w_i\ |\ t) \times \sum_{t'} p(t\ |\ t') \times \alpha(i - 1, t') $
- $ \beta(n, t) = 1 $
- $ \beta(i-1, t) = \sum_{t'} p(w_{i}\ |\ t') \times p(t'\ |\ t) \times \beta(i, t') $

Intuitively,
- **Forward beliefs** $\alpha(i, t)$ represent the sum of all the paths that end in tag $t$ at position $i$.
- **Backward beliefs** $\beta(i, t)$ represent the sum of all the ways to continue on from tag $t$ at position $i$ through to position $n$.

If we combine the forward beliefs (information from before position $i$) with the backward beliefs (information from beyond position $i$), we get the exact probability distribution:

$$ p(y_i = t\ |\ x) = \alpha(i,t) \times \beta(i,t) $$

### Log-probabilities

Note that we're multiplying a lot of probabilities together in the above equations. While each term is easy to represent as a float, we can quickly run into numerical underflow issues when multiplying together a large number of small values.  If your dataset is as small as this and the sentences you want to tag are short (i.e. not work with real data), you can sometimes get away without worrying about this.  Alternatively, you can petition Intel to improve precision of floating point numbers close to 0.

To avoid this, we'll perform all our calculations in log space. This means that we start with the log-probabilities (as computed by `HMM.compute_logprobs()`), and replace multiplication with addition and addition with log-sum-exp, according to the identities:

- $ \log(a b) = \log(a) + \log(b) $
- $ \log(a + b) = \log(e^{\log(a)} + e^{\log(b)}) = \text{LogSumExp}(log(a), log(b)) $

To implement the latter, we recommend using [`scipy.misc.logsumexp`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.misc.logsumexp.html), which is imported for you in the starter code.

#### Cheat Sheet:  Summing probabilities

Add probabilities together, $P(t_1) + P(t_2) + \cdots + P(t_n)$.
```python
# "Regular" Probabilities
sum_py = sum([p[t] for t in tags])
# Log-probabilities
log_sum_py = logsumexp([logp[t] for t in tags])
```

At the end of running this code,
- `sum_py` $ = \Sigma P(t_i)$
- `log_sum_py` $ = log(\Sigma P(t_i))$

Normal and log-probabilities can always be converted into each other with a $e^x$ or $log(x)$, although you shouldn't need to do this explicitly in this assignment.

_**Hint:**_ Your code in this part should look a lot like the math. In particular, `self.initial(...)`, `self.transition(...)`, and `self.emission(...)` are functions that are already set up to return appropriate defaults ($\log p(...) = \log 0 = -\infty$) for unseen tags - so you shouldn't need to check membership with `if` statements or `dict.get()`.

### Questions
1.  Implement `forward()` (i.e. compute $\alpha$) in `pos.py`
2.  Inspect the code for `forward_backward()` in `pos.py`.
3.  What does forward/backward do at a high level?
4.  How does this manifest in the equations above?
5.  What can you say about the sequence of tags it produces?

### Answers to Forward-Backward questions
(Your answers to Q3-5.)

#### Answer 3).
`To make a best guess on the PoS tag of a particular position in a sentence, we consider a set of tag candidates. For each tag candidate, forward-backward sums up sequence probabilities from all sequences that contain the tag candidate at the position of interest. It then compares this score across all tag candidates and pick the tag with the highest score. `

`The actual implementation is less direct than the overall idea sounds. For each tag candidate t at position i, the algorithm first split the sentence at position i. Then, the algorithm looks up both forward believe up to and including position i with tag t and backward believe from position i with tag t to the end. The product of forward and backward believes then gives us the sum of sequence probabilities from all sequences that contain the tag candidate t at position i.`



#### Answer 4).
`For each tag candidate t at position i, forward believe alpha(i,t) gives the total of sequence probabilities (including transmission and emission) that leads up to and including position i with tag t, while beta(i,t) gives the total of sequence probabilities that carries from position i with tag t to the end. alpha(0,t_0) concerns the start of sequence thus is initiated by product of probability of the tag t (rather than a transmission probability) and emission probability of word w_0 given t_0. beta(n,t_n) concerns the end of sequence thus is initiated by 1 -- there is only one way to carry forward from the end. Finally, the probability distribution of the possible tag labels at position i is derived from computing P(y_i=t|x)  for each t in the tag set, that is, the product of corresponding forward believe alpha(i,t) and backward believe beta(i,t) for each t.`



#### Answer 5).
`The sequence of tags it produces may not be coherent/ makes most grammar sense because individual tags are optimized at each position. This sequence probability can be lower than the best sequence probability from a more coherent/English like tag sequence.`


In [3]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_alpha"])

test_alpha (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [4]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_beta"])

test_beta (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [5]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_forward_backward"])

test_forward_backward (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## Let's give it a try!

(Warning: if you decide to try some of your own - and you should! - you may find the limited vocabulary of the training set to be problematic.)

In [6]:
reload(pos)
hmm = pos.HMM()
for sentence in corpus.tagged_sents():
    hmm.update_counts(sentence)
hmm.compute_logprobs()
def pretty_print_fb(sentence):
    print(sentence)
    print(hmm.forward_backward(sentence.split()))

In [7]:
pretty_print_fb('Pierre will join the board .')
pretty_print_fb('Pierre joined an organization .')
pretty_print_fb('The old man .')

Pierre will join the board .
['NNP', 'MD', 'VB', 'DT', 'NN', '.']
Pierre joined an organization .
['NNP', 'VBD', 'DT', 'NN', '.']
The old man .
['DT', 'JJ', 'NN', '.']


In [12]:
pretty_print_fb('the apple')
pretty_print_fb('Pierre ate the apple')

the apple
['DT', 'NN']
Pierre ate the apple
['``', '``', '``', '``']


In [10]:
pretty_print_fb('The horse raced pass the barn fell.')
pretty_print_fb('I ate an apple')
pretty_print_fb('Parking permit a parking spot')

The horse raced pass the barn fell.
['``', '``', '``', '``', '``', '``', '``']
I ate an apple
['``', '``', '``', '``']
Parking permit a parking spot
['``', '``', '``', '``', '``']


## Part (c): Viterbi

Viterbi finds the maximum likelihood sequence of assignments, rather than considering a single assignment at a time.  Its implementation is a small tweak on the $\alpha$ of Forward Backward.  In particular, instead of trying to find the _sum_ of all the possible ways to make a part of speech in a particular position, we try to find the _best_ way. Formally, we have:

- $\delta(0, t) = p(t) \times P(x_0\ |\ t)$
- $\delta(i, t) = p(x_i\ |\ t) \times \max_{t'} \left[\delta(i - 1, t') \times p(t\ |\ t')\right]$

_**Hint:**_ As in part (b), your code should look quite a lot like the math above.

### Questions:
1.  Read the `viterbi()` function at the bottom of `pos.py`.  It uses the `delta` table and backpointers to determine the most likely sequence of part of speech tags.
2.  Implement the equations immediately above pos.py's `build_viterbi_delta()`.
3.  What does Viterbi do differently in its algorithm than forward of forward/backward?
4.  What does this mean for the tags it produces?

### Answers to Viterbi questions
(Your answers to Q3 and Q4.)

#### answers to Q3 

`Both algorithm pre-compute partial sequence probabilities. Viterbi computes from left to right and store them in a single detla table while forward/backward computes from both left to right and right to left. The left to right forward believes are stored in the alpha table and the right to left backward believes are stored in the beta table.`

`At inference time, since Viterbi only concerns the single sequence with best probabilities, the algorithm uses the back-pointer table to track the best tag for every position. Since Forward/backward concerns for each position the tag that accumulates most probabilities to the left and right of it, the algorithm computes for each tag its sum of forward (to the left) and backward (to the right) believes from the alpha and beta tables. The tag with the highest sum is selected.`

#### answers to Q4
`The tags it produces for a sequence are likely to be coherent and comply with English grammar, because all tags come from the same best sequence.`


In [18]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_build_viterbi_delta"])

test_build_viterbi_delta (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [19]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_viterbi_end_to_end"])

test_viterbi_end_to_end (pos_test.TestTagging) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Let's give it a try!

(Warning: if you decide to try some of your own - and you should! - you may find the limited vocabulary of the training set to be problematic.)

In [20]:
corpus = nltk.corpus.treebank
# Uncomment below if you install the full Penn Treebank
# corpus = nltk.corpus.ptb

# Optional: set to true to get a nice progressbar during training.
use_fancy_progressbar = True
if use_fancy_progressbar:
    utils.require_package("tqdm")
    from tqdm import tqdm as ProgressBar
else:
    ProgressBar = lambda x: x  # pass-through iterator

reload(pos)
hmm = pos.HMM()
for sentence in ProgressBar(corpus.tagged_sents()):
    hmm.update_counts(sentence)
hmm.compute_logprobs()

100%|██████████| 3914/3914 [00:01<00:00, 2645.02it/s]


In [21]:
def pretty_print_v(sentence):
    tokens = sentence.split()
    tags = hmm.viterbi(tokens)
    print(" ".join("%s/%s" % (w,t) for (w,t) in zip(tokens, tags)))

pretty_print_v('Pierre will join the board .')
pretty_print_v('Pierre joined an organization .')
pretty_print_v('The old man .')

Pierre/NNP will/MD join/VB the/DT board/NN ./.
Pierre/NNP joined/VBD an/DT organization/NN ./.
The/DT old/JJ man/NN ./.


In [22]:
pretty_print_fb('the apple')
pretty_print_fb('Pierre ate the apple')

the apple
['DT', 'NN']
Pierre ate the apple
['``', '``', '``', '``']


In [23]:
pretty_print_fb('The horse raced pass the barn fell.')
pretty_print_fb('I ate an apple')
pretty_print_fb('Parking permit a parking spot')

The horse raced pass the barn fell.
['``', '``', '``', '``', '``', '``', '``']
I ate an apple
['``', '``', '``', '``']
Parking permit a parking spot
['``', '``', '``', '``', '``']
